###Mount Drive


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


### Install Libraries

In [2]:
!pip install diffusers==0.8.0
!pip install transformers scipy ftfy
!pip install ipywidgets==7.7.2
!pip install flask==1.1.1 werkzeug==0.15.6 itsdangerous==2.0.1 jinja2==3.0.3
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.1/246.1 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 82.6 MB/s eta 0:00:00
  Attempting uninstall: jupyterlab-widgets
    Found existing 

### Import Libraries

In [3]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
from PIL import Image
from flask import Flask, render_template, redirect, url_for, request,session,Response
from werkzeug import secure_filename
import os
import cv2
from pyngrok import ngrok

###Load Diffusion Model


In [4]:
experimental_pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=False) 
experimental_pipe = experimental_pipe.to("cuda")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


###App Code

In [5]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [6]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://enr8dca1xrg-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [9]:
app = Flask(__name__,template_folder='templates')
ngrok.set_auth_token("2MPTP3WIZZHf6VPQdrbRSMBBwes_5eCXCYsULaob8Drhy3Sms")
public_url =  ngrok.connect(5000).public_url
print(public_url)
app.secret_key = '1234'
app.config["CACHE_TYPE"] = "null"
app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 0

@app.route('/home', methods=['GET', 'POST'])
def home():
	return redirect(url_for('input'))

@app.route('/', methods=['GET', 'POST'])
def input():
	error = None
	global login_status
	if request.method == 'POST':
		if request.form['username'] != 'admin' or request.form['password'] != 'admin':
			error = 'Invalid Credentials. Please try again.'
		else:
			login_status = 1
			return redirect(url_for('image'))

	return render_template('input.html', error=error)


@app.route('/image', methods=['GET', 'POST'])
def image():
	global result
	if request.method == 'POST':
		if request.form['sub']=='Generate':
			text = request.form['text']

			description_1 = text
			with autocast("cuda"):
				image_1 = experimental_pipe(description_1).images[0]  
				# Now to display an image you can do either save it such as:
				image_1.save(f"/content/drive/MyDrive/Colab Notebooks/static/images/generated.png")
				#cv2.imwrite('static/images/generated.png',image_1)

			return render_template('image.html',text=text)
	return render_template('image.html')

@app.route('/result', methods=['GET', 'POST'])
def result():
	return render_template('result.html')

# No caching at all for API endpoints.
@app.after_request
def add_header(response):
	# response.cache_control.no_store = True
	response.headers['Cache-Control'] = 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0, max-age=0'
	response.headers['Pragma'] = 'no-cache'
	response.headers['Expires'] = '-1'
	return response


if __name__ == '__main__':
	app.run(port=5000)

http://5c57-35-204-54-22.ngrok-free.app
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [11/Apr/2023 16:09:28] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Apr/2023 16:09:30] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Apr/2023 16:09:37] "POST / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [11/Apr/2023 16:09:38] "GET /image HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Apr/2023 16:09:40] "GET /static/images/generated.png HTTP/1.1" 200 -


  0%|          | 0/51 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [11/Apr/2023 16:10:17] "POST /image HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Apr/2023 16:10:18] "GET /static/images/generated.png HTTP/1.1" 200 -
